## **Establish connection with GitHub using SSH protocol.**

In [1]:
# Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# ssh keys were generated earlier. Private and Public keys are stored in 
# 'colabkey' and 'colabkey.pub' files. Additionally a config file is also 
# stored in /content/drive/MyDrive/ssh_keys/ on google drive.

# Remove ssh folder and its contents if already present
!rm -rf /root/.ssh

# Create a directory
!mkdir /root/.ssh

# Copy everything (ssh_key files & config file) from google drive to Virtual Machine.  
!cp /content/drive/MyDrive/ssh_keys/* /root/.ssh

# Set permission
!chmod 700 /root/.ssh  

In [3]:
# Add the git server as an ssh known host
!touch /root/.ssh/known_hosts

# Trust github  
!ssh-keyscan github.com >> /root/.ssh/known_hosts

# Set permission  
!chmod 644 /root/.ssh/known_hosts 

# github.com:22 SSH-2.0-babeld-ea6f11ef
# github.com:22 SSH-2.0-babeld-ea6f11ef
# github.com:22 SSH-2.0-babeld-ea6f11ef
# github.com:22 SSH-2.0-babeld-ea6f11ef
# github.com:22 SSH-2.0-babeld-ea6f11ef


In [4]:
# Run ssh-agent and add `ssh-add /root/.ssh/colabkey` in the prompt. 
# After this command, optinally check if the key is saved using `ssh-add -l`, 
# then exit the prompt.
!ssh-agent /bin/bash

bash: cannot set terminal process group (145): Inappropriate ioctl for device
bash: no job control in this shell
/content# ssh-add /root/.ssh/colabkey
Identity added: /root/.ssh/colabkey (shreyasdb99@gmail.com)
/content# exit
exit


In [5]:
# Check connection with github
!ssh -T git@github.com

Hi shre-db! You've successfully authenticated, but GitHub does not provide shell access.


In [6]:
# Navigate to the project folder  # Please note that you need to remove this line of comment while running the cd command below.
cd drive/MyDrive/ColabProjects/stock-price-prediction-using-RNN-LSTM

/content/drive/MyDrive/ColabProjects/stock-price-prediction-using-RNN-LSTM


In [7]:
!ls

notebook.ipynb


## **Initialize git repository if not already**

In [8]:
# Initialize the directory as a git repository (if not already)
!git init

Initialized empty Git repository in /content/drive/MyDrive/ColabProjects/stock-price-prediction-using-RNN-LSTM/.git/


In [9]:
# Configure user name and email (if not already)
!git config user.name "shre-db"
!git config user.email "shreyasdb99@gmail.com"

In [10]:
# Check Staging Area
!git status

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	notebook.ipynb

nothing added to commit but untracked files present (use "git add" to track)


In [ ]:
# Add Commit and Push
!echo "# Stock Price Prediction using RNN-LSTM" >> README.md
!git add README.md notebook.ipynb
!git commit -m "Add README and notebook" -m "This commit includes a README file and a notebook containing code for preliminary setup."
!git branch -M main
!git remote add origin git@github.com:shre-db/stock-price-prediction-using-RNN-LSTM.git
!git push -u origin main

## **Use cells in this section to execute commands**

In [ ]:
!git status 

In [ ]:
!git add notebook.ipynb
!git commit -m " " -m " "
!git push

## **Working with Data**